# Monte-Carlo

Основан на оценке значения состояний с помощью среднего взвешенного значения наград, полученных в ходе эпизодов взаимодействия агента со средой. Он относится к методам оценки значения (Value Estimation), которые сосредотачиваются на оценке функции значения и оптимизации на основе этой оценки.

Основные шаги метода Монте-Карло:


1. Задаем некоторое первое приближение политики $\pi_0$
2. Задаем количество эпизодов моделирования $N$.
3. Задаем некоторое текущее приближение политики $\pi$ (на первом шаге будет равно $\pi_0)$
3. Инициализируем массивы $Q(S, A) = 0$ и $N(S, A) = 0$
4. Задаем количество итераций в траектории: $K$ и для каждой итерации (т.е в цикле для каждого $k \in [1, K]$) выполняем:
    - согласно текущей политики $\pi$ проходим по среде и получаем траекторию $\tau=(S_0, S_1, \dots S_T)$, функцию раграды $R_{\tau} = (R_0, R_1, \dots, R_{T-1})$ и суммарную награду по траектории $G_{\tau} = (G_0, G_1, \dots, G_{T-1}))$.
    - возвращаемся в начало траектории и движемся по ней на каждом шаге $t$ обновляя оценку функции $Q(S_t, A_t) \gets Q(S_t, A_t) + \frac{1}{N(S_t, A_t)+1}(G_t - Q(S_t, A_t))$. 
    - проходим в цикле по всем $k$.
5. Обновляем политику выбирая для каждого состояния ту аппроксимацию функции $Q(s, a)$ которая дает наибольший прирост суммарной награды и с некоторой вероятностью $\epsilon$ другую аппроксимацию, что обеспечивает блуждание и исследоание системы на наличие новых состояний.
6. Возвращаемся в начало в п.3 в качестве приближения политики используя текущую политику $\pi$
7. Проходим по в цикле по всем эпизодам $n \in N$

Фактически мы получаем внешний цикл по эпизодам и внутри вложенные циклы: цикл по траекториям, потом цикл по расчету награды на каждом шаге и затем цикл по обновлению фнукции $Q$.

In [ ]:
def MonteCarlo(env, episode_n, trajectory_len=500, gamma=0.99):
    total_rewards = []  # Создаем список для хранения общих вознаграждений для каждого эпизода
    
    state_n = env.observation_space.n  # Получаем количество состояний в среде
    action_n = env.action_space.n  # Получаем количество действий в среде
    qfunction = np.zeros((state_n, action_n))  # Создаем Q-функцию (матрицу состояние-действие) и инициализируем её нулями
    counter = np.zeros((state_n, action_n))  # Создаем счетчик для подсчета количества визитов в каждую ячейку Q-функции
    
    for episode in range(episode_n):  # Запускаем цикл для каждого эпизода
        epsilon = 1 - episode / episode_n  # Уменьшаем параметр epsilon для epsilon-жадной стратегии с каждым эпизодом
        trajectory = {'states': [], 'actions': [], 'rewards': []}  # Создаем структуру данных для хранения траектории эпизода
        
        state = env.reset()[0]  # Сбрасываем среду и получаем начальное состояние
        for _ in range(trajectory_len):  # Запускаем цикл для каждого шага внутри эпизода (ограниченного trajectory_len)
            trajectory['states'].append(state)  # Добавляем текущее состояние в траекторию
            
            action = get_epsilon_greedy_action(qfunction[state], epsilon, action_n)  # Получаем действие с использованием epsilon-жадной стратегии
            trajectory['actions'].append(action)  # Добавляем текущее действие в траекторию
            
            state, reward, done, _, _ = env.step(action)  # Выполняем выбранное действие и получаем следующее состояние, вознаграждение и флаг завершения
            trajectory['rewards'].append(reward)  # Добавляем полученное вознаграждение в траекторию
            
            if done:  # Если эпизод завершился, выходим из цикла
                break
                
        total_rewards.append(sum(trajectory['rewards']))  # Добавляем суммарное вознаграждение текущего эпизода в список
        
        real_trajectory_len = len(trajectory['rewards'])  # Определяем реальную длину траектории (может быть меньше trajectory_len, если эпизод завершился раньше)
        returns = np.zeros(real_trajectory_len + 1)  # Создаем массив для хранения возвращений (returns) на каждом шаге траектории
        
        for t in range(real_trajectory_len - 1, -1, -1):  # Запускаем цикл для вычисления возвращений для каждого шага в траектории (в обратном порядке)
            returns[t] = trajectory['rewards'][t] + gamma * returns[t + 1]  # Вычисляем возвращение с учетом дисконтирования (gamma)
            
        for t in range(real_trajectory_len):  # Запускаем цикл для обновления Q-функции на каждом шаге в траектории
            state = trajectory['states'][t]  # Получаем текущее состояние
            action = trajectory['actions'][t]  # Получаем текущее действие
            qfunction[state][action] += (returns[t] - qfunction[state][action]) / (1 + counter[state][action])  # Обновляем Q-функцию согласно формуле метода Монте-Карло
            counter[state][action] += 1  # Увеличиваем счетчик визитов для соответствующей ячейки Q-функции
            
    return total_rewards  # Возвращаем список общих вознаграждений для каждого эпизода

# SARSA

Основные шаги метода SARSA:

1. Устанавливаем количество эпизодов $N$.
2. Внутри эпизода начинаем проходить по шагам траектории $t$ и на каждом шаге:
    - Выполняем выбранное действие и получаем следующее состояние, вознаграждение и флаг завершения
    - Получаем следующее действие с использованием epsilon-жадной стратегии
    - Обновляем Q-функцию с использованием следующего состояния и следующего действия __согласно политики__ (отличие от алгоритма q-learning)
    - Добавляем полученное вознаграждение к общему вознаграждению текущего эпизода
3. Завершаем эпизод, обновляем $\epsilon$ и переходим к следующему эпизоду.

$\alpha$ - шиперпараметр имеющий смысл скорости обучения. $\gamma$ - гиперпараметр имеющий смысл коэффициента дисконтирования, т.е определяющего что более ценно награда сейчас или награда будущих шагов.

In [ ]:
def SARSA(env, episode_n, gamma=0.99, trajectory_len=500, alpha=0.5):
    total_rewards = np.zeros(episode_n)  # Создаем массив для хранения общих вознаграждений для каждого эпизода
    
    state_n = env.observation_space.n  # Получаем количество состояний в среде
    action_n = env.action_space.n  # Получаем количество действий в среде
    qfunction = np.zeros((state_n, action_n))  # Создаем Q-функцию (матрицу состояние-действие) и инициализируем её нулями
    
    for episode in range(episode_n):  # Запускаем цикл для каждого эпизода
        epsilon = 1 / (episode + 1)  # Уменьшаем параметр epsilon для epsilon-жадной стратегии с каждым эпизодом
        
        state = env.reset()[0]  # Сбрасываем среду и получаем начальное состояние
        action = get_epsilon_greedy_action(qfunction[state], epsilon, action_n)  # Получаем действие с использованием epsilon-жадной стратегии
        
        for _ in range(trajectory_len):  # Запускаем цикл для каждого шага внутри эпизода (ограниченного trajectory_len)
            next_state, reward, done, _, _ = env.step(action)  # Выполняем выбранное действие и получаем следующее состояние, вознаграждение и флаг завершения
            next_action = get_epsilon_greedy_action(qfunction[next_state], epsilon, action_n)  # Получаем следующее действие с использованием epsilon-жадной стратегии
            
            qfunction[state][action] += alpha * (reward + gamma * qfunction[next_state][next_action] - qfunction[state][action])  # Обновляем Q-функцию согласно формуле метода SARSA
            
            state = next_state  # Переходим в следующее состояние
            action = next_action  # Переходим в следующее действие
            
            total_rewards[episode] += reward  # Добавляем полученное вознаграждение к общему вознаграждению текущего эпизода
            
            if done:  # Если эпизод завершился, выходим из цикла
                break

    return total_rewards  # Возвращаем массив общих вознаграждений для каждого эпизода

# Q-learning

Основные шаги метода Q-learning:

1. Устанавливаем количество эпизодов $N$.
2. Внутри эпизода начинаем проходить по шагам траектории $t$ и на каждом шаге:
    - Выполняем выбранное действие и получаем следующее состояние, вознаграждение и флаг завершения
    - Получаем следующее действие с использованием epsilon-жадной стратегии
    - Обновляем Q-функцию с использованием следующего состояния и __максимального__ Q-значения для следующего состояния (отличие от алгоритма SARSA). Т.е берем строку со всеми возможными значениями Q функции в следующем состоянии, т.е от всех возможных
    - Добавляем полученное вознаграждение к общему вознаграждению текущего эпизода
3. Завершаем эпизод, обновляем $\epsilon$ и переходим к следующему эпизоду.

$\alpha$ - шиперпараметр имеющий смысл скорости обучения. $\gamma$ - гиперпараметр имеющий смысл коэффициента дисконтирования, т.е определяющего что более ценно награда сейчас или награда будущих шагов.

In [ ]:
def QLearning(env, episode_n, noisy_episode_n, gamma=0.99, t_max=500, alpha=0.5):
    state_n = env.observation_space.n  # Получаем количество состояний в среде
    action_n = env.action_space.n  # Получаем количество действий в среде

    Q = np.zeros((state_n, action_n))  # Создаем Q-таблицу размерности (количество состояний) x (количество действий) и инициализируем нулями
    epsilon = 1  # Значение параметра epsilon для epsilon-greedy стратегии, которое будет уменьшаться по мере обучения

    total_rewards = []  # Список для хранения наград в каждом эпизоде
    for episode in range(episode_n):  # Начинаем обучение на заданное количество эпизодов

        total_reward = 0  # Суммарная награда в текущем эпизоде
        state = env.reset()  # Сбрасываем среду в начальное состояние и получаем начальное состояние

        for t in range(t_max):  # В каждом эпизоде продолжаем обучение в течение максимального количества временных шагов t_max
            action = get_epsilon_greedy_action(Q[state], epsilon, action_n)  # Получаем действие с использованием epsilon-greedy стратегии на основе текущей Q-таблицы

            next_state, reward, done, _ = env.step(action)  # Выполняем выбранное действие и получаем следующее состояние, вознаграждение и флаг завершения

            Q[state][action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state][action])  # Обновляем значение Q-таблицы для текущего состояния и выбранного действия на основе полученной награды и максимального Q-значения для следующего состояния

            total_reward += reward  # Увеличиваем суммарную награду для текущего эпизода

            if done:  # Если агент достиг целевого состояния (done=True), заканчиваем эпизод
                break

            state = next_state  # Переходим в следующее состояние для следующего временного шага

        epsilon = max(0, epsilon - 1 / noisy_episode_n)  # Уменьшаем epsilon с течением времени для уменьшения исследования в пользу использования текущей стратегии

        total_rewards.append(total_reward)  # Добавляем суммарную награду текущего эпизода в список total_rewards

    return total_rewards  # Возвращаем список суммарных наград для каждого эпизода обучения
